In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
lm=LinearRegression()
%matplotlib inline
from sklearn.cluster import KMeans

Location="loansmall.csv"
rawdata = pd.read_csv(Location)
workdata=rawdata

Copied the data into a new dataframe.

In [ ]:
#I use this so I can do a quick check when I want to.
workdata.head()

In [ ]:
# getting an overview of our data
print("Our data has {0} rows and {1} columns".format(workdata.shape[0], workdata.shape[1]))
# checking for missing values
print("Are there missing values? {}".format(workdata.isnull().any().any()))

In [2]:
workdata=workdata.dropna(axis=1,how='all')

Dropped the columns with completely missing data

In [3]:
workdata=workdata.drop('url',axis=1)
workdata=workdata.drop('desc',axis=1)
workdata=workdata.drop('application_type',axis=1)
workdata=workdata.drop('policy_code',axis=1)
workdata=workdata.drop('earliest_cr_line',axis=1)
workdata=workdata.drop('title',axis=1)
workdata=workdata.drop('sub_grade',axis=1)
workdata=workdata.drop('home_ownership',axis=1)
workdata=workdata.drop('verification_status',axis=1)
workdata=workdata.drop('loan_status',axis=1)


Dropped the columns with no variation, columns with no apparent quantifiable purpose, and columns I no longer need.

In [ ]:
headers=workdata.columns
headers

In [4]:
#Changed term length to a numerical value. Only two possibilities so I used a binary option
def term_to_num(x):
    if x == ' 36 months':
        return 0
    if x == ' 60 months':
        return 1
workdata['term']=rawdata['term'].apply(term_to_num)

In [5]:
#I am using sub grades to get a better spectrum of values.
def grade_to_num(x):
    if x == 'G5':
        return 0
    if x == 'G4':
        return 1
    if x == 'G3':
        return 2
    if x == 'G2':
        return 3
    if x == 'G1':
        return 4
    if x == 'F5':
        return 5
    if x == 'F4':
        return 6
    if x == 'F3':
        return 7
    if x == 'F2':
        return 8
    if x == 'F1':
        return 9
    if x == 'E5':
        return 10
    if x == 'E4':
        return 11
    if x == 'E3':
        return 12
    if x == 'E2':
        return 13
    if x == 'E1':
        return 14
    if x == 'D5':
        return 15
    if x == 'D4':
        return 16
    if x == 'D3':
        return 17
    if x == 'D2':
        return 18
    if x == 'D1':
        return 19
    if x == 'C5':
        return 20
    if x == 'C4':
        return 21
    if x == 'C3':
        return 22
    if x == 'C2':
        return 23
    if x == 'C1':
        return 24
    if x == 'B5':
        return 25
    if x == 'B4':
        return 26
    if x == 'B3':
        return 27
    if x == 'B2':
        return 28
    if x == 'B1':
        return 29
    if x == 'A5':
        return 30
    if x == 'A4':
        return 31
    if x == 'A3':
        return 32
    if x == 'A2':
        return 33
    if x == 'A1':
        return 34

#expdata=workdata
workdata['grade']=rawdata['sub_grade'].apply(grade_to_num)

In [7]:
#Changing employment length to numerical values.
def emp_length_to_num(x):
    if x == '< 1 year':
        return 0
    if x == '1 year':
        return 1
    if x == '2 years':
        return 2
    if x == '3 years':
        return 3
    if x == '4 years':
        return 4
    if x == '5 years':
        return 5
    if x == '6 years':
        return 6
    if x == '7 years':
        return 7
    if x == '8 years':
        return 8
    if x == '9 years':
        return 9
    if x == '10+ years':
        return 10
#expdata=workdata
workdata['emp_length']=rawdata['emp_length'].apply(emp_length_to_num)
#workdata['emp_length']

In [6]:
#Home Ownership. I did it this way because I can't see putting levels to this data, so I used dummy columns.
def rent_to_num(x):
    if x == 'RENT':
        return 1
    else:
        return 0

def mortgage_to_num(x):
    if x == 'MORTGAGE':
        return 1
    else:
        return 0
    
def none_to_num(x):
    if x == 'NONE':
        return 1
    else:
        return 0
    
def other_to_num(x):
    if x == 'OTHER':
        return 1
    else:
        return 0
    
def own_to_num(x):
    if x == 'OWN':
        return 1
    else:
        return 0
#expdata=workdata
workdata['home_rent']=rawdata['home_ownership'].apply(rent_to_num)
workdata['home_mortgage']=rawdata['home_ownership'].apply(mortgage_to_num)
workdata['home_none']=rawdata['home_ownership'].apply(none_to_num)
workdata['home_other']=rawdata['home_ownership'].apply(other_to_num)
workdata['home_own']=rawdata['home_ownership'].apply(own_to_num)
#expdata.head()
#workdata['emp_length']

In [8]:
#Verification Status. I did it this way because I can't see putting levels to this data, so I used dummy columns.
def verified_to_num(x):
    if x == 'Verified':
        return 1
    else:
        return 0

def not_verified_to_num(x):
    if x == 'Not Verified':
        return 1
    else:
        return 0
    
def source_verified_to_num(x):
    if x == 'Source Verified':
        return 1
    else:
        return 0
#expdata=workdata
workdata['verified']=rawdata['verification_status'].apply(verified_to_num)
workdata['not_verified']=rawdata['verification_status'].apply(not_verified_to_num) 
workdata['source_verified']=rawdata['verification_status'].apply(source_verified_to_num)

In [9]:
def pmnt_plan_to_num(x):
    if x == 'n':
        return 0
    if x == 'y':
        return 1
workdata['pymnt_plan']=rawdata['pymnt_plan'].apply(pmnt_plan_to_num)

In [10]:
#loan Status. I did it this way because I can't see putting levels to this data, so I used dummy columns.
def charged_off_to_num(x):
    if x == 'Charged Off' or x == 'Does not meet the credit policy. Status:Charged Off':
        return 1
    else:
        return 0

def default_to_num(x):
    if x == 'Default' :
        return 1
    else:
        return 0
    
def fully_paid_to_num(x):
    if x == 'Fully Paid' or x == 'Does not meet the credit policy. Status:Fully Paid' :
        return 1
    else:
        return 0

#expdata=workdata
workdata['status_charged_off']=rawdata['loan_status'].apply(charged_off_to_num)
workdata['status_default']=rawdata['loan_status'].apply(default_to_num)
workdata['status_fully_paid']=rawdata['loan_status'].apply(fully_paid_to_num)

In [25]:
def initial_list_status_to_num(x):
    if x == 'f':
        return 0
    if x == 'w':
        return 1
workdata['initial_list_status']=rawdata['initial_list_status'].apply(initial_list_status_to_num)

In [26]:
#I am using this for testing what I am doing. It is basically pointless for the overall project
pd.pivot_table(workdata, index= 'initial_list_status', values= "member_id", aggfunc='count', fill_value= 0,
              margins = True, margins_name= 'Total')

,member_id
initial_list_status,
0,154383
1,39728
Total,194111


In [12]:
workdata.corr()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,home_mortgage,home_none,home_other,home_own,verified,not_verified,source_verified,status_charged_off,status_default,status_fully_paid
id,1.000000,0.970585,0.125117,0.135422,0.160821,0.033971,0.167846,0.144287,-0.043744,0.115888,...,0.056126,-0.015179,-0.039888,0.022904,0.070280,-0.143011,0.077355,-0.062776,0.013659,-0.338658
member_id,0.970585,1.000000,0.125686,0.135927,0.161444,0.034337,0.159657,0.143200,-0.036344,0.120372,...,0.058607,-0.015383,-0.040281,0.022678,0.071933,-0.138172,0.069914,-0.062453,0.014227,-0.336929
loan_amnt,0.125117,0.125686,1.000000,0.996551,0.986320,0.409893,0.193517,0.950588,-0.179311,0.141528,...,0.205722,-0.000769,-0.011722,-0.037158,0.341707,-0.378923,0.018904,0.013073,0.006114,-0.123544
funded_amnt,0.135422,0.135927,0.996551,1.000000,0.990482,0.404719,0.195963,0.955838,-0.178170,0.142903,...,0.205189,-0.000655,-0.012813,-0.036713,0.341113,-0.378677,0.019335,0.011600,0.006376,-0.126808
funded_amnt_inv,0.160821,0.161444,0.986320,0.990482,1.000000,0.406978,0.199897,0.945451,-0.169027,0.149866,...,0.205138,-0.001759,-0.020027,-0.035222,0.344228,-0.387185,0.025214,0.003593,0.006936,-0.132907
term,0.033971,0.034337,0.409893,0.404719,0.406978,1.000000,0.433965,0.163739,-0.449931,0.088845,...,0.115957,-0.004081,-0.015304,-0.018012,0.270246,-0.338290,0.058539,0.081295,0.008944,-0.182948
int_rate,0.167846,0.159657,0.193517,0.195963,0.199897,0.433965,1.000000,0.179737,-0.958313,0.030431,...,-0.103745,0.001797,-0.003931,0.011819,0.187726,-0.205710,0.007606,0.158405,0.017563,-0.183943
installment,0.144287,0.143200,0.950588,0.955838,0.945451,0.163739,0.179737,1.000000,-0.156061,0.125274,...,0.167374,0.001082,-0.009325,-0.031139,0.299002,-0.324774,0.008871,0.007255,0.005979,-0.092019
grade,-0.043744,-0.036344,-0.179311,-0.178170,-0.169027,-0.449931,-0.958313,-0.156061,1.000000,-0.001333,...,0.107973,-0.001463,-0.006726,-0.006886,-0.164422,0.181645,-0.008323,-0.174508,-0.014032,0.140742
emp_length,0.115888,0.120372,0.141528,0.142903,0.149866,0.088845,0.030431,0.125274,-0.001333,1.000000,...,0.208549,0.000763,-0.013027,0.016384,0.107870,-0.099758,-0.015177,-0.014304,0.006394,-0.073009


In [15]:
#Going to run a regression analysis to determine fully_paid_status of a loan
result = sm.ols(formula='status_fully_paid ~ out_prncp + out_prncp_inv + total_rec_prncp + last_pymnt_amnt + status_charged_off -1', data=workdata).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      status_fully_paid   R-squared:                       0.626
Model:                            OLS   Adj. R-squared:                  0.626
Method:                 Least Squares   F-statistic:                 6.491e+04
Date:                Tue, 23 Oct 2018   Prob (F-statistic):               0.00
Time:                        19:04:07   Log-Likelihood:            -1.1908e+05
No. Observations:              194111   AIC:                         2.382e+05
Df Residuals:                  194106   BIC:                         2.382e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
out_prncp             -0.0009   8.29e-05    -11.304      0.000      -0.001      -0.001
out_prncp_inv          0.0009   8.29e-05     10.866      0.000       0.001       0.001
total_rec_prncp     3.872e-05   1.36e-07    284.512      0.000    3.85e-05     3.9e-05
last_pymnt_amnt     2.071e-05   2.47e-07     83.772      0.000    2.02e-05    2.12e-05
status_charged_off    -0.1612      0.003    -54.641      0.000      -0.167      -0.155
==============================================================================
Omnibus:                     3919.894   Durbin-Watson:                   1.511
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2360.958
Skew:                          -0.115   Prob(JB):                         0.00
Kurtosis:                       2.511   Cond. No.                     4.07e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#Going to run a regression analysis to determine default_status of a loan
result = sm.ols(formula='status_default ~ ', data=workdata).fit()
result.summary()

#The stuff below this comment is not finished. I probably won't be able to finish until Thursday.

In [27]:
defaultdata=workdata

In [28]:
defaultdata = defaultdata.drop('id',axis=1)
defaultdata = defaultdata.drop('member_id',axis=1)
defaultdata = defaultdata.drop('emp_title',axis=1)
defaultdata = defaultdata.drop('issue_d',axis=1)
defaultdata = defaultdata.drop('purpose',axis=1)
defaultdata = defaultdata.drop('zip_code',axis=1)
defaultdata = defaultdata.drop('addr_state',axis=1)
defaultdata = defaultdata.drop('last_pymnt_d',axis=1)
defaultdata = defaultdata.drop('last_pymnt_amnt',axis=1)
defaultdata = defaultdata.drop('next_pymnt_d',axis=1)
defaultdata = defaultdata.drop('last_credit_pull_d',axis=1)
defaultdata = defaultdata.drop('status_charged_off',axis=1)
defaultdata = defaultdata.drop('status_fully_paid',axis=1)


In [29]:
X = defaultdata.drop('status_default', axis = 1)
lm.fit(X, defaultdata.status_default)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
#Going to run a regression analysis to determine default_status of a loan
result = sm.ols(formula='status_default ~ ', data=workdata).fit()
result.summary()